In [ ]:
dir = '/content/drive/MyDrive/Project/MLB/Data'

## 1. Download the youtube videos

In [ ]:
# !git clone https://github.com/piergiaj/mlb-youtube.git
!pip install youtube-dl
!pip install yt-dlp

In [ ]:
import os
import json
import string
import random
import subprocess
from tqdm.notebook import tqdm
from IPython.display import clear_output

save_dir = dir + '/All'
targ_fil = dir + '/data_describe/mlb-youtube-segmented.json'

with open(targ_fil, 'r') as f:
    data = json.load(f)

    for entry in tqdm(data):
        yturl = data[entry]['url']
        ytid = yturl.split('=')[-1]

        # print(yturl, ytid)

        if os.path.exists(os.path.join(save_dir, ytid+'.mp4')):
            continue

        cmd = 'yt-dlp -f 22 '+yturl+' -o ' + os.path.join(save_dir, ytid+'.mp4')
        # os.system(cmd)
        !{cmd}
        clear_output(wait=True)
        # cmd = ['yt-dlp', '-f', '22', '-o', os.path.join(save_dir, ytid+'.mp4')]
        # result = subprocess.run(cmd, stdout=subprocess.PIPE)
        # print(result.stdout.decode('utf-8'))
        

[youtube] xcNCZK_g4_A: Downloading webpage
[youtube] xcNCZK_g4_A: Downloading android player API JSON
[info] xcNCZK_g4_A: Downloading 1 format(s): 22
[download] Destination: /content/drive/MyDrive/Project/MLB/Data/All/xcNCZK_g4_A.mp4
[download] 100% of 3.06GiB in 01:13                 


## 2. To extract the segmented video clips

In [ ]:
!apt upgrade ffmpeg
!ffmpeg -version

In [ ]:
import os
import json
import string
import random
import subprocess
import multiprocessing

def local_clip(filename, start_time, duration, output_filename, output_directory):
    end_time = start_time + duration
    command = ['ffmpeg',
               '-i', '"%s"' % filename,
               '-ss', str(start_time),
               '-t', str(end_time - start_time),
               '-c:v', 'copy', '-an',
               '-threads', '1',
               '-loglevel', 'panic',
               os.path.join(output_directory, output_filename)]
    command = ' '.join(command)

    try:
        output = subprocess.check_output(command, shell=True,
                                         stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as err:
        print(err.output)
        return err.output


def wrapper(clip):
    input_directory = dir + '/All'
    output_directory = dir + '/Segment'

    if 'swing' in clip['labels']:
        output_directory = dir + '/Segment/Swing/'
    else:
        output_directory = dir + '/Segment/Others/'
    


    file_name = clip['url'].split('=')[1] + '_' + str(clip['start'])
    duration = clip['end'] - clip['start']
    filename = clip['url'].split('=')[-1]

    if os.path.exists(os.path.join(input_directory, filename + '.mp4')):
        return 0

    local_clip(os.path.join(input_directory, filename + '.mp4'), 
              clip['start'], duration, 
              f'{file_name}.mp4', output_directory)
    return 0


targ_fil = dir + '/data_describe/mlb-youtube-segmented.json'

with open(targ_fil, 'r') as f:
    data = json.load(f)
    pool = multiprocessing.Pool(processes=8)
    datas = [data[k] for k in data.keys()]
    pool.map(wrapper, datas)

print(f'Number of videos: {len(datas)}')

In [ ]:
import os

now_dir = dir + '/Segment/' + 'Swing/'

files = os.listdir(now_dir)
file_count = len(files)

print(f'Swing: {file_count}')


now_dir = dir + '/Segment/' + 'Others/'

files = os.listdir(now_dir)
file_count = len(files)

print(f'Other: {file_count}')

Swing: 2676
Other: 3130


## 3. To extract the continuous video clips

# 4. Dataset Processing


In [ ]:
idx = 0
for i in lis_otr:
    num = '{:0>4}'.format(idx)
    idx += 1
    ori_nam = i
    now_nam = f'{num}_otr.mp4'
    shutil.move(f'{otr_path}/{ori_nam}', f'{tar_path}/{now_nam}')

In [ ]:
idx = 0
for i in lis_swg:
    num = '{:0>4}'.format(idx)
    idx += 1
    ori_nam = i
    now_nam = f'{num}_swg.mp4'
    shutil.move(f'{swg_path}/{ori_nam}', f'{tar_path}/{now_nam}')

In [ ]:
import os
import shutil

dir = '/content/drive/MyDrive/Project/MLB/Data/Segment'
tar_path = dir + '/all_test_video'

lis_all = os.listdir(tar_path)
for i in lis_all:
    if 'swg' in i:
        shutil.move(f'{tar_path}/{i}', f'{tar_path}/Swing/{i}')
        # print(f'{tar_path}/{i}', f'{tar_path}/Swing/{i}')
    if 'otr' in i:
        shutil.move(f'{tar_path}/{i}', f'{tar_path}/Others/{i}')
        # print(f'{tar_path}/{i}', f'{tar_path}/Other/{i}')

In [ ]:
import os
import shutil

dir = '/content/drive/MyDrive/Project/MLB/Data/Segment/all_test_video'

otr_path = dir + '/Others'
swg_path = dir + '/Swing'
tar_path = dir + '/all_test_video'

lis_swg = os.listdir(swg_path)
lis_otr = os.listdir(otr_path)

print(f'Swing: {len(lis_swg)}')
print(f'Others: {len(lis_otr)}')
# shutil.move('/home/user/Downloads/test.txt', '/home/user/Documents/useful_name.txt')

Swing: 2676
Others: 3130


In [ ]:
import os
import sys
import shutil
from tqdm.notebook import tqdm

path = '/content/drive/MyDrive/Project/MLB/Data/Segment/all_test/Swing'
tmp = os.listdir(path)
tmp = sorted(tmp)

print(f'Now swing size = {len(tmp)}')

cnt = 0
for i in tqdm(tmp):
    now = os.listdir(f'{path}/{i}')
    if len(now) == 0:
        cnt += 1
        # print(f'{i}')
        # shutil.rmtree(f'{path}/{i}')

print(f'Number of video Less than 1 frame: {cnt}')

Now swing size = 2676


  0%|          | 0/2676 [00:00<?, ?it/s]

Number of video Less than 1 frame: 55


In [ ]:
import os
import cv2
import threading
from queue import Queue

"""
Given individual video files (mp4, webm) on disk, creates a folder for
every video file and saves the video's RGB frames as jpeg files in that
folder.

It can be used to turn SomethingSomethingV2, which comes as 
many ".webm" files, into an RGB folder for each ".webm" file.
Uses multithreading to extract frames faster.

Modify the two filepaths at the bottom and then run this script.
"""

def video_to_rgb(video_filename, out_dir, resize_shape):
    file_template = 'frame_{0:012d}.jpg'
    reader = cv2.VideoCapture(video_filename)
    success, frame, = reader.read()  # read first frame
    count = 0
    while success:
        out_filepath = os.path.join(out_dir, file_template.format(count))
        # do resize on transform is enough
        #frame = cv2.resize(frame, resize_shape)
        cv2.imwrite(out_filepath, frame)
        success, frame = reader.read()
        count += 1
    reader.release()

def process_videofile(video_filename, video_path, rgb_out_path, file_extension: str ='.mp4'):
    filepath = os.path.join(video_path, video_filename)
    video_filename = video_filename.replace(file_extension, '')
    out_dir = os.path.join(rgb_out_path, video_filename)
    # Already Processed
    if os.path.isdir(out_dir):
        return 
    os.mkdir(out_dir)
    video_to_rgb(filepath, out_dir, resize_shape=(299, 299))


def thread_job(queue, video_path, rgb_out_path, file_extension='.mp4'):
    while not queue.empty():
        video_filename = queue.get()
        process_videofile(video_filename, video_path, rgb_out_path, file_extension=file_extension)
        queue.task_done()



dir = '/content/drive/MyDrive/Project/MLB/Data/Segment'

file_extension = '.mp4'
src_path = dir + '/all_test_video'
dst_path = dir + '/all_test'

for src in os.listdir(src_path):
    # the path to the folder which contains all video files (mp4, webm, or other)
    video_path = os.path.join(src_path, src)
    # the root output path where RGB frame folders should be created
    rgb_out_path = os.path.join(dst_path, src)
    # if the folder didnt exist yet
    if not os.path.isdir(rgb_out_path):
        os.mkdir(rgb_out_path)
        print(f'New label found! {src}')

    # the file extension that the videos have
    video_filenames = []
    all_filenames = os.listdir(video_path)
    don_filenames = os.listdir(rgb_out_path)
    for i in sorted(all_filenames):
        if i.split('.')[0] not in don_filenames:
            video_filenames.append(i)
    # print(len(don_filenames))
    # print(len(video_filenames))
    # break

    queue = Queue()
    [queue.put(video_filename) for video_filename in sorted(video_filenames)]
    
    NUM_THREADS = 4
    for i in range(NUM_THREADS):
        worker = threading.Thread(target=thread_job, args=(queue, video_path, rgb_out_path, file_extension))
        worker.start()

    print('Waiting for all videos to be completed.', queue.qsize(), 'videos')
    print('This can take few hours depending on the size of dataset')
    queue.join()
    print('All done')

Waiting for all videos to be completed. 0 videos
This can take few hours depending on the size of dataset
All done
Waiting for all videos to be completed. 244 videos
This can take few hours depending on the size of dataset
All done


In [ ]:
# Generate annotaion file
import os
from tqdm.notebook import tqdm

dir = '/content/drive/MyDrive/Project/MLB/Data/Segment'
dst_path = dir + '/all_test'

annotation = []
label_id = 0

for src in os.listdir(dst_path):
    test_path = os.path.join(dst_path, src)
    if not os.path.isdir(test_path):
        continue
    print(f'Doing on {src}, label = {label_id}')
    all_filenames = os.listdir(test_path)

    for i in tqdm(sorted(all_filenames)):
        now_files = os.listdir(os.path.join(test_path, f'{i}'))
        if len(now_files) == 0:
            continue
        annotation.append(f'{src}/{i} 1 {len(now_files)} {label_id}')    
    label_id += 1

# print(annotation)

Doing on Swing, label = 0


  0%|          | 0/2676 [00:00<?, ?it/s]

Doing on Others, label = 1


  0%|          | 0/3130 [00:00<?, ?it/s]

In [ ]:
print(f'Writing annotation.txt, size = {len(annotation)}')
with open(f'{dst_path}/annotations.txt', 'w') as f:
    for item in annotation:
        f.write("%s\n" % item)

Writing annotation.txt, size = 5685


# Generate Small Dataset

In [4]:
# Generate annotaion file
import os
from tqdm.notebook import tqdm

dir = '/content/drive/MyDrive/Project/MLB/Data/Segment'
dst_path = dir + '/all_test'

annotation = []
label_id = 0

for src in os.listdir(dst_path):
    test_path = os.path.join(dst_path, src)
    if not os.path.isdir(test_path):
        continue
    print(f'Doing on {src}, label = {label_id}')
    all_filenames = os.listdir(test_path)

    max_size = 200
    for i in tqdm(sorted(all_filenames)):
        now_files = os.listdir(os.path.join(test_path, f'{i}'))
        max_size -= 1
        if max_size == 0:
            break   
        if len(now_files) == 0:
            continue
        if src == 'Swing' and len(now_files) >= 45:
            continue
        now_files = sorted(now_files)
        beg_numb = int(now_files[0].split('_')[1].split('.')[0])
        annotation.append(f'{src}/{i} {beg_numb} {beg_numb + len(now_files) - 1} {label_id}')


    label_id += 1

print(f'Total size = {len(annotation)}')

Doing on Swing, label = 0


  0%|          | 0/2676 [00:00<?, ?it/s]

Doing on Others, label = 1


  0%|          | 0/2977 [00:00<?, ?it/s]

Total size = 328


In [6]:
print(f'Writing annotation_small.txt, size = {len(annotation)}')
with open(f'{dst_path}/annotations_small.txt', 'w') as f:
    for item in annotation:
        f.write("%s\n" % item)

Writing annotation_small.txt, size = 328


In [8]:
import random
TEST_RATIO = 0.3

def split_dataset():
    dir = '/content/drive/MyDrive/Project/MLB/Data/Segment'
    dst_path = dir + '/all_test'

    all = []

    with open(f'{dst_path}/annotations_small.txt', 'r') as f:
        for item in f:
            all.append(item.strip())

    n = int(len(all) * TEST_RATIO)
    test = random.sample(all, n)
    tran = []
    for i in all:
        if i not in test:
            tran.append(i)

    print(f'Total dataset size = {len(all)}')
    print(f'==================================')
    print(f'Train size = {len(tran)}')
    print(f'Test  size = {len(test)}')

    with open(f'{dst_path}/annotations_train_small.txt', 'w') as f:
        for item in tran:
            f.write("%s\n" % item)

    with open(f'{dst_path}/annotations_test_small.txt', 'w') as f:
        for item in test:
            f.write("%s\n" % item)


split_dataset()

Total dataset size = 328
Train size = 230
Test  size = 98


## Others

[MLB-YouTube Dataset](https://github.com/piergiaj/mlb-youtube)

In [ ]:
!pip install speedtest-cli
!speedtest-cli

Retrieving speedtest.net configuration...
Testing from Google Cloud (34.86.80.20)...
Retrieving speedtest.net server list...
Selecting best server based on ping...
Hosted by PhoenixNAP Global IT Services (Ashburn, VA) [42.41 km]: 2.765 ms
Testing download speed................................................................................
Download: 1412.54 Mbit/s
Testing upload speed......................................................................................................
Upload: 910.16 Mbit/s
